In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import cv2
import keras
from keras.preprocessing import image
from keras.engine import Layer
from keras.layers import Conv2D, Conv3D, UpSampling2D, InputLayer, Conv2DTranspose, Input, Reshape, merge, concatenate
from keras.layers import Activation, Dense, Dropout, Flatten
from keras.layers.normalization import BatchNormalization
from keras.callbacks import TensorBoard
from keras.models import Sequential, Model
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, rgb2gray, gray2rgb
from skimage.transform import resize
from skimage.io import imsave
from time import time
import numpy as np
import os
import random
import tensorflow as tf
from PIL import Image, ImageFile
import cv2


In [ ]:
images_gray = np.load('/kaggle/input/image-colorization/l/gray_scale.npy')
images_lab = np.load('/kaggle/input/image-colorization/ab/ab/ab1.npy')

In [ ]:
image_gray = images_gray[0]
image_lab = images_lab[0]

In [ ]:

# Initializing with zeros ( or any random number)
img = np.zeros((224, 224, 3))
img[:, :, 0] = image_gray
img[:, :, 1:] = image_lab

# Changing the data type of the img array to 'uint8'
# Refer the above markdown for it.
img = img.astype('uint8')

In [ ]:
img_ = cv2.cvtColor(img, cv2.COLOR_LAB2RGB)


print('Gray scale image')
plt.imshow(image_gray)
plt.show()

print('Recreated image')
plt.imshow(img_)
plt.show()

In [ ]:
X=[]
Y=[]
for i in range(0,2000):
    image_gray = images_gray[i]
    image_lab = images_lab[i]
    img = np.zeros((224, 224, 3))
    img[:, :, 0] = image_gray
    img[:, :, 1:] = image_lab
    img = img.astype('uint8')
    img_ = cv2.cvtColor(img, cv2.COLOR_LAB2RGB)
    new_array=cv2.resize(img_,(256,256))
    new_array = new_array.astype('float32')
    new_array /= 255
    lab = rgb2lab(new_array)
    try:
        lab = rgb2lab(new_array)
        X.append(lab[:,:,0])
        Y.append(lab[:,:,1:] / 128)
    except:
        print('error')
    

In [ ]:
X = np.array(X)
Y = np.array(Y)
X = X.reshape(X.shape+(1,))
print(X.shape)
print(Y.shape)

In [ ]:

encoder_input = Input(shape=(256, 256, 1,))
encoder_output = Conv2D(64, (3,3),strides=(2, 2) ,activation='relu', padding='same')(encoder_input)
encoder_output = Conv2D(128, (3,3),strides=(1, 1), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(128, (3,3), strides=(2, 2),activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(256, (3,3),strides=(1, 1), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(256, (3,3),strides=(2, 2), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(512, (3,3), strides=(1, 1),activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(512, (3,3), strides=(1, 1),activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(256, (3,3),strides=(1, 1) ,activation='relu', padding='same')(encoder_output)

#Decoder
decoder_output = Conv2D(128, (3,3), activation='relu', padding='same')(encoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
decoder_output = Conv2D(64, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
decoder_output = Conv2D(32, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = Conv2D(16, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = Conv2D(2, (3, 3), activation='tanh', padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
model = Model(inputs=encoder_input, outputs=decoder_output)

In [ ]:
X.shape

In [ ]:
print(model.summary())
model.compile(optimizer='adam', loss='mse' , metrics=['accuracy'])
t=model.fit(X,Y,validation_split=0.2,batch_size=2, epochs=500,verbose=1)
print(t.history.keys())

In [ ]:
import matplotlib.pyplot as plt
plt.plot(t.history['accuracy'])
plt.plot(t.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(t.history['loss'])
plt.plot(t.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()


In [ ]:
test = '/kaggle/input/building/pexels-photo-1105766.jpeg'
#test = os.listdir(test_path)
for imgName in test:
    color_me = []
    img = img_to_array(load_img(test))
    img = resize(img ,(256,256))
    color_me.append(img)
color_me = np.array(color_me, dtype=float)
color_me = rgb2lab(1.0/255*color_me)[:,:,:,0]
color_me = color_me.reshape(color_me.shape+(1,))
print(color_me.shape)
output = model.predict(color_me)
output = output * 128
# Output colorizations
for i in range(len(output)):
    result = np.zeros((256, 256, 3))
    result[:,:,0] = color_me[i][:,:,0]
    result[:,:,1:] = output[i]
    imsave("result1.png", lab2rgb(result))
    x=lab2rgb(result)
    plt.imshow(x)
    plt.show()